In [1]:
import torch
import pickle
from models import mobilenetv2, resnet56
from torchvision.models import resnet50
from embedl.plumbing.torch.metrics.target import Target
from embedl.torch.pruning.methods import UniformPruning
from embedl.torch.viewer import view_model
from embedl.torch.metrics.performances import Flops  
from embedl.torch.metrics.measure_performance import measure_flops
import torchvision.datasets as datasets
from embedl.torch.pruning.methods import plot_pruning_profile 
import torchvision.transforms as transforms
import torch.nn as nn
from embedl.torch.metrics.performances import Flops
from embedl.torch.pruning.methods import (
    PruningMethod,
    ChannelPruningTactic,
)
from embedl.plumbing.torch.metrics.scorers import ChannelPruningScorer, PruningBalancer
from embedl.torch.metrics.importance_scores import WeightMagnitude


normalize = transforms.Normalize(
    mean=[0.4914, 0.4822, 0.4465], std=[0.247, 0.243, 0.261]
)

val_loader = torch.utils.data.DataLoader(
    datasets.CIFAR10(
        root="/home/jonna/data",
        train=False,
        transform=transforms.Compose(
            [
                transforms.ToTensor(),
                normalize,
            ]
        ),
    ),
    batch_size=128,
    shuffle=False,
    num_workers=2,
    pin_memory=True,
)

def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


def validate(val_loader, model, criterion):
    """
    Run evaluation
    """

    prec1 = 0
    count = 0
    # switch to evaluate mode
    model.eval()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
            target = target.cuda()
            input_var = input.cuda()
            target_var = target.cuda()

            # compute output
            output = model(input_var)
            loss = criterion(output, target_var)

            output = output.float()
            loss = loss.float()

            # measure accuracy and record loss
            prec1 += accuracy(output.data, target)[0] * target.size(0)
            # print(accuracy(output.data, target)[0])
            count += target.size(0)

    print(f" * Prec@1 {prec1/count:.3f}")
    return

In [7]:
model = torch.load('/home/jonna/hyperparameter_sensitivity_pruning/experiments/imagenet/base_model/results/lr_10**-1.00_wd_10**-4.00/checkpoint.pth')
model['model']
r = resnet50()
r.load_state_dict(model['model'])

<All keys matched successfully>

In [6]:
r

<All keys matched successfully>

In [8]:
input_shape = [1, 3, 224, 224]
print(measure_flops(model=r, input_shape=input_shape)
)
pruning_method = UniformPruning(target=Target(Flops(), fraction=0.4), step_size=1)
pruning_steps = pruning_method.prune(r, input_shape)
pruned_flops = measure_flops(model=r, input_shape=input_shape)


Found nodes that do not have (<class 'embedl.plumbing.torch.pruning.node_configs.base.MaskChannelPruningConfig'>,) implemented: 

    0. input node (1 occurrence)
    1. <class 'torch.nn.modules.activation.ReLU'> (49 occurrences)
    2. <class 'torch.nn.modules.pooling.MaxPool2d'> (1 occurrence)
    3. <built-in function add> (16 occurrences)
    4. <class 'torch.nn.modules.pooling.AdaptiveAvgPool2d'> (1 occurrence)
    5. <built-in method flatten of type object at 0x7f3e715fd460> (1 occurrence)
    6. output node (1 occurrence)

This may limit certain functionality. Extend the dict returned by MaskChannelPruning.node_config_map() to extend support to these.


8242043856


  0%|          | [01:20<12:27:05, 74816.33s/it] (Flops)  

KeyboardInterrupt: 